In [3]:
!pip install numpy opencv-python
!pip install transformers torch

In [4]:
import pandas as pd
import os
import cv2
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import multiprocessing
import matplotlib.pyplot as plt
from transformers import BertTokenizer, BertModel
import torch
from sklearn.decomposition import PCA
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras import regularizers, layers, optimizers, callbacks

In [56]:
expanded_p10_sampled_df = pd.read_csv("/kaggle/input/sampled-dataset/expanded_p10_sampled_df.csv")
expanded_p10_sampled_df

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,image_id
0,10000032.0,50414267.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg
1,10000032.0,50414267.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962.jpg
2,10000032.0,53189527.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab.jpg
3,10000032.0,53189527.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c.jpg
4,10000764.0,57375967.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,b79e55c3-735ce5ac-64412506-cdc9ea79-f1af521f.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15110,10498557.0,59910967.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,f4182731-0ec785ba-682d16cf-7ce9ee64-4e4bcf4e.jpg
15111,10498557.0,52457766.0,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6178ea01-4993f9b9-e12d7878-dba186dc-c2eaa58b.jpg
15112,10498557.0,52457766.0,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47673283-5d296d28-e7439649-021b778f-2b30963f.jpg
15113,10498687.0,53217608.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,c4cc1e86-e9fa862b-058f961d-685868bd-48216d06.jpg


In [57]:
expanded_p10_sampled_df['subject_id'] = expanded_p10_sampled_df['subject_id'].astype(float).astype(int).astype(str)
expanded_p10_sampled_df['study_id'] = expanded_p10_sampled_df['study_id'].astype(float).astype(int).astype(str)
print(expanded_p10_sampled_df[['subject_id', 'study_id']].head())

  subject_id  study_id
0   10000032  50414267
1   10000032  50414267
2   10000032  53189527
3   10000032  53189527
4   10000764  57375967


In [58]:
expanded_p10_sampled_df

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,image_id
0,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg
1,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962.jpg
2,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab.jpg
3,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c.jpg
4,10000764,57375967,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,b79e55c3-735ce5ac-64412506-cdc9ea79-f1af521f.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15110,10498557,59910967,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,f4182731-0ec785ba-682d16cf-7ce9ee64-4e4bcf4e.jpg
15111,10498557,52457766,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6178ea01-4993f9b9-e12d7878-dba186dc-c2eaa58b.jpg
15112,10498557,52457766,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47673283-5d296d28-e7439649-021b778f-2b30963f.jpg
15113,10498687,53217608,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,c4cc1e86-e9fa862b-058f961d-685868bd-48216d06.jpg


In [59]:
unique_study_ids = expanded_p10_sampled_df['study_id'].unique()
print(f"Number of unique study_ids in expanded_p10_sampled_df: {len(unique_study_ids)}")

print(unique_study_ids)

Number of unique study_ids in expanded_p10_sampled_df: 9521
['50414267' '53189527' '57375967' ... '59910967' '52457766' '53217608']


In [9]:
report_df = pd.read_csv("/kaggle/input/report-dataset/full_report_data.csv")
report_df.head()

,subject_id,study_id,examination,indication,technique,comparison,findings,impression
0,p18726783,s54132939,Evaluation of the patient after ICD placement ...,NaN,NaN,NaN,NaN,NaN
1,p18726783,s54838151,NaN,"NSTEMI, residual shortness of breath, changes ...",NaN,___.,NaN,NaN
2,p18726783,s55647339,"Evaluation of the patient with nausea, dysuria.",NaN,NaN,NaN,NaN,NaN
3,p18726783,s55239487,NaN,NaN,PA and lateral views of the chest.,___ chest radiograph ___ chest CTA.,The heart remains moderately enlarged. The ao...,"Small bilateral pleural effusions, left greate..."
4,p18726783,s53461826,Evaluation of the patient with congestive heart,NaN,NaN,NaN,NaN,NaN


In [10]:
report_df['subject_id'] = report_df['subject_id'].str.lstrip('p')  # Remove 'p' from subject_id
report_df['study_id'] = report_df['study_id'].str.lstrip('s')      # Remove 's' from study_id

print(report_df.head())

  subject_id  study_id                                        examination  \
0   18726783  54132939  Evaluation of the patient after ICD placement ...   
1   18726783  54838151                                                NaN   
2   18726783  55647339    Evaluation of the patient with nausea, dysuria.   
3   18726783  55239487                                                NaN   
4   18726783  53461826    Evaluation of the patient with congestive heart   

                                          indication  \
0                                                NaN   
1  NSTEMI, residual shortness of breath, changes ...   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                            technique                           comparison  \
0                                 NaN                                  NaN   
1                                 NaN       

In [11]:
subset_report_df = report_df[report_df['subject_id'].str.startswith('10')]
print(subset_report_df)

      subject_id  study_id                    examination  \
45624   10867166  58802645            CHEST (PORTABLE AP)   
45625   10867166  52151611                            NaN   
45626   10867166  52713612            CHEST (PORTABLE AP)   
45627   10867166  50570766  Portable AP chest radiograph.   
45628   10867166  59572179            CHEST (PORTABLE AP)   
...          ...       ...                            ...   
67816   10180823  57889799                            NaN   
67817   10879723  58557418                            NaN   
67818   10879723  56907357            CHEST (PORTABLE AP)   
67819   10879723  55139535             CHEST (PA AND LAT)   
67820   10879723  53426859             CHEST (PA AND LAT)   

                                              indication  \
45624  ___ year old man with ? IABP position adjustme...   
45625  History: ___M with recent STEMI and PTCA here ...   
45626  ___ year old man with sCHF, UTI now w/ wheezin...   
45627  ___ year old man wit

In [12]:
subset_report_df.value_counts

<bound method DataFrame.value_counts of       subject_id  study_id                    examination  \
45624   10867166  58802645            CHEST (PORTABLE AP)   
45625   10867166  52151611                            NaN   
45626   10867166  52713612            CHEST (PORTABLE AP)   
45627   10867166  50570766  Portable AP chest radiograph.   
45628   10867166  59572179            CHEST (PORTABLE AP)   
...          ...       ...                            ...   
67816   10180823  57889799                            NaN   
67817   10879723  58557418                            NaN   
67818   10879723  56907357            CHEST (PORTABLE AP)   
67819   10879723  55139535             CHEST (PA AND LAT)   
67820   10879723  53426859             CHEST (PA AND LAT)   

                                              indication  \
45624  ___ year old man with ? IABP position adjustme...   
45625  History: ___M with recent STEMI and PTCA here ...   
45626  ___ year old man with sCHF, UTI now w/ w

In [13]:
unique_study_ids = expanded_p10_sampled_df['study_id'].unique()
p10_sampled_reports_df = report_df[report_df['study_id'].isin(unique_study_ids)]

print("Filtered DataFrame with unique study_ids from expanded_p10_sampled_df:")
print(p10_sampled_reports_df)

Filtered DataFrame with unique study_ids from expanded_p10_sampled_df:
      subject_id  study_id                           examination  \
45634   10032890  50259406                                   NaN   
45642   10003052  58630288                                   NaN   
45644   10136491  59612844  DX CHEST PORT LINE/TUBE PLCMT 1 EXAM   
45645   10136491  58132816                                   NaN   
45647   10482402  51258466                                   NaN   
...          ...       ...                                   ...   
67804   10481162  58541326                                   NaN   
67805   10004457  55439624                                   NaN   
67808   10026404  54766817                                   NaN   
67815   10119094  52202735                    CHEST (PA AND LAT)   
67816   10180823  57889799                                   NaN   

                                              indication  \
45634    ___M with chest pain  // Eval for acute pro

In [14]:
def load_and_clean_data(df):
    text_columns = ['examination', 'indication', 'technique', 'comparison', 'findings', 'impression']
    for col in text_columns:
        df[col] = df[col].replace(['0', '___', 'None.', None], 'missing')
    
    return df
final_reports_df = load_and_clean_data(p10_sampled_reports_df)
print(final_reports_df.head())  


      subject_id  study_id                           examination  \
45634   10032890  50259406                               missing   
45642   10003052  58630288                               missing   
45644   10136491  59612844  DX CHEST PORT LINE/TUBE PLCMT 1 EXAM   
45645   10136491  58132816                               missing   
45647   10482402  51258466                               missing   

                                              indication  \
45634    ___M with chest pain  // Eval for acute process   
45642  ___-year-old man with advanced dementia and 3 ...   
45644  ___ year old woman with gastric mass, with GI ...   
45645                  Concern for aspiration pneumonia.   
45647                                            missing   

                                               technique  \
45634                               Chest PA and lateral   
45642                               Chest PA and lateral   
45644                                            m

/tmp/ipykernel_23/2536131752.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].replace(['0', '___', 'None.', None], 'missing')


In [15]:
def replace_missing_with_unk(df):
    missing_values = ['None', '___', '0', None, 'NaN', 'missing']
    for col in df.columns:
        df[col] = df[col].replace(missing_values, '[UNK]')
    return df

final_reports_df = replace_missing_with_unk(final_reports_df)
print(final_reports_df)

      subject_id  study_id                           examination  \
45634   10032890  50259406                                 [UNK]   
45642   10003052  58630288                                 [UNK]   
45644   10136491  59612844  DX CHEST PORT LINE/TUBE PLCMT 1 EXAM   
45645   10136491  58132816                                 [UNK]   
45647   10482402  51258466                                 [UNK]   
...          ...       ...                                   ...   
67804   10481162  58541326                                 [UNK]   
67805   10004457  55439624                                 [UNK]   
67808   10026404  54766817                                 [UNK]   
67815   10119094  52202735                    CHEST (PA AND LAT)   
67816   10180823  57889799                                 [UNK]   

                                              indication  \
45634    ___M with chest pain  // Eval for acute process   
45642  ___-year-old man with advanced dementia and 3 ...   
456

/tmp/ipykernel_23/4154608064.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].replace(missing_values, '[UNK]')


In [16]:
unique_study_ids_reports = final_reports_df['study_id'].unique()
print(f"Number of unique study_ids in final_reports_df: {len(unique_study_ids_reports)}")

print(unique_study_ids_reports)

Number of unique study_ids in final_reports_df: 9521
['50259406' '58630288' '59612844' ... '54766817' '52202735' '57889799']


In [17]:
expanded_counts = expanded_p10_sampled_df['study_id'].value_counts()
duplicate_study_ids = expanded_counts[expanded_counts > 1]
duplicated_rows = []

for study_id, count in duplicate_study_ids.items():
    row_to_duplicate = final_reports_df[final_reports_df['study_id'] == study_id]
    if not row_to_duplicate.empty:
        duplicated_rows.extend([row_to_duplicate.iloc[0]] * (count - 1))
duplicated_rows_df = pd.DataFrame(duplicated_rows)

final_reports_with_duplicates = pd.concat([final_reports_df, duplicated_rows_df], ignore_index=True)
print("Original final_reports_df shape:", final_reports_df.shape)
print("Duplicated final_reports_df shape:", final_reports_with_duplicates.shape)
print("Value counts of study_ids after duplication:")
print(final_reports_with_duplicates['study_id'].value_counts())

Original final_reports_df shape: (9521, 8)
Duplicated final_reports_df shape: (15115, 8)
Value counts of study_ids after duplication:
study_id
55370162    6
57285901    5
55034075    5
56492208    5
53910326    5
           ..
55355476    1
53100450    1
55505534    1
54353128    1
50101371    1
Name: count, Length: 9521, dtype: int64


In [18]:
final_reports_with_duplicates

,subject_id,study_id,examination,indication,technique,comparison,findings,impression
0,10032890,50259406,[UNK],___M with chest pain // Eval for acute process,Chest PA and lateral,[UNK],Heart size is normal. There is mild unfolding...,No acute cardiopulmonary process.
1,10003052,58630288,[UNK],___-year-old man with advanced dementia and 3 ...,Chest PA and lateral,[UNK],The lung volumes are low. The lungs are clear...,Clear lungs.
2,10136491,59612844,DX CHEST PORT LINE/TUBE PLCMT 1 EXAM,"___ year old woman with gastric mass, with GI ...",[UNK],[UNK],[UNK],"In comparison with the study of ___, the right..."
3,10136491,58132816,[UNK],Concern for aspiration pneumonia.,Frontal and lateral chest radiographs.,Radiograph from ___.,[UNK],A right IJ central venous catheter terminates ...
4,10482402,51258466,[UNK],[UNK],Frontal and lateral chest radiographs were obt...,Chest radiograph ___ and ___.,An opacity in the lingula with mild volume los...,No evidence of acute cardiopulmonary process.
...,...,...,...,...,...,...,...,...
15110,10401318,51172492,Sign rib O,___-year-old man with fever. Evaluate for pne...,Chest PA and lateral,Chest radiograph dated ___.,No significant interval change. Lungs are wel...,No pneumonia.
15111,10401318,59030330,[UNK],___ year old man with f/u PNA // compare with...,Chest PA and lateral,[UNK],The previously noted right lower lobe pneumoni...,The previously noted right lower lobe pneumoni...
15112,10401318,56447172,[UNK],"History: ___M with PMH splenectomy, hodgkins l...",Chest: Frontal and Lateral,Chest radiograph from ___.,"There is right lower lobe opacity, concerning ...",Right lower lobe pneumonia.\n \n NOTIFICATION:...
15113,10402217,50609320,[UNK],[UNK],[UNK],[UNK],[UNK],PA and lateral chest compared to ___:\n \n Nor...


In [19]:
final_reports_with_duplicates = final_reports_with_duplicates.sort_values(by='subject_id', ascending=True)
print(final_reports_with_duplicates.head())

      subject_id  study_id         examination  \
7317    10000032  50414267  CHEST (PA AND LAT)   
7316    10000032  53189527  CHEST (PA AND LAT)   
13690   10000032  53189527  CHEST (PA AND LAT)   
13689   10000032  50414267  CHEST (PA AND LAT)   
11160   10000764  57375967  CHEST (PA AND LAT)   

                                              indication  \
7317   ___F with new onset ascites  // eval for infec...   
7316              History: ___F with shortness of breath   
13690             History: ___F with shortness of breath   
13689  ___F with new onset ascites  // eval for infec...   
11160            ___M with hypoxia  // ?pna, aspiration.   

                  technique comparison  \
7317   Chest PA and lateral      [UNK]   
7316   Chest PA and lateral      [UNK]   
13690  Chest PA and lateral      [UNK]   
13689  Chest PA and lateral      [UNK]   
11160                 [UNK]      [UNK]   

                                                findings  \
7317   There is no focal 

In [21]:
final_reports_with_duplicates = final_reports_with_duplicates.reset_index(drop=True)
print(final_reports_with_duplicates.head())

  subject_id  study_id         examination  \
0   10000032  50414267  CHEST (PA AND LAT)   
1   10000032  53189527  CHEST (PA AND LAT)   
2   10000032  53189527  CHEST (PA AND LAT)   
3   10000032  50414267  CHEST (PA AND LAT)   
4   10000764  57375967  CHEST (PA AND LAT)   

                                          indication             technique  \
0  ___F with new onset ascites  // eval for infec...  Chest PA and lateral   
1             History: ___F with shortness of breath  Chest PA and lateral   
2             History: ___F with shortness of breath  Chest PA and lateral   
3  ___F with new onset ascites  // eval for infec...  Chest PA and lateral   
4            ___M with hypoxia  // ?pna, aspiration.                 [UNK]   

  comparison                                           findings  \
0      [UNK]  There is no focal consolidation, pleural effus...   
1      [UNK]  The cardiac, mediastinal and hilar contours ar...   
2      [UNK]  The cardiac, mediastinal and hilar cont

In [22]:
final_reports_with_duplicates

,subject_id,study_id,examination,indication,technique,comparison,findings,impression
0,10000032,50414267,CHEST (PA AND LAT),___F with new onset ascites // eval for infec...,Chest PA and lateral,[UNK],"There is no focal consolidation, pleural effus...",No acute cardiopulmonary process.
1,10000032,53189527,CHEST (PA AND LAT),History: ___F with shortness of breath,Chest PA and lateral,[UNK],"The cardiac, mediastinal and hilar contours ar...",No acute cardiopulmonary abnormality.
2,10000032,53189527,CHEST (PA AND LAT),History: ___F with shortness of breath,Chest PA and lateral,[UNK],"The cardiac, mediastinal and hilar contours ar...",No acute cardiopulmonary abnormality.
3,10000032,50414267,CHEST (PA AND LAT),___F with new onset ascites // eval for infec...,Chest PA and lateral,[UNK],"There is no focal consolidation, pleural effus...",No acute cardiopulmonary process.
4,10000764,57375967,CHEST (PA AND LAT),"___M with hypoxia // ?pna, aspiration.",[UNK],[UNK],PA and lateral views of the chest provided. ...,"Focal consolidation at the left lung base, pos..."
...,...,...,...,...,...,...,...,...
15110,10498557,59910967,[UNK],[UNK],[UNK],[UNK],[UNK],[UNK]
15111,10498557,52457766,Chest PA and lateral.,___-year-old female patient with cellulitis an...,[UNK],[UNK],PA and lateral chest views were obtained with ...,Mild progression of cardiac enlargement with m...
15112,10498557,52457766,Chest PA and lateral.,___-year-old female patient with cellulitis an...,[UNK],[UNK],PA and lateral chest views were obtained with ...,Mild progression of cardiac enlargement with m...
15113,10498687,53217608,CHEST (PA AND LAT),___ year old woman with history of left rib in...,[UNK],Chest radiographs ___.,[UNK],"Normal heart, lungs, hila, mediastinum, and pl..."


In [23]:
final_reports_with_duplicates['combined_text'] = final_reports_with_duplicates['examination'] + ' ' + \
                                                  final_reports_with_duplicates['indication'] + ' ' + \
                                                  final_reports_with_duplicates['technique'] + ' ' + \
                                                  final_reports_with_duplicates['comparison'] + ' ' + \
                                                  final_reports_with_duplicates['findings'] + ' ' + \
                                                  final_reports_with_duplicates['impression']


print(final_reports_with_duplicates[['subject_id', 'combined_text']].head())


  subject_id                                      combined_text
0   10000032  CHEST (PA AND LAT) ___F with new onset ascites...
1   10000032  CHEST (PA AND LAT) History: ___F with shortnes...
2   10000032  CHEST (PA AND LAT) History: ___F with shortnes...
3   10000032  CHEST (PA AND LAT) ___F with new onset ascites...
4   10000764  CHEST (PA AND LAT) ___M with hypoxia  // ?pna,...


In [47]:
tokenizer = BertTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
model = BertModel.from_pretrained("dmis-lab/biobert-v1.1")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


def get_embeddings_without_unk(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    token_embeddings = outputs.last_hidden_state
    token_ids = inputs['input_ids'][0]
    unk_token_id = tokenizer.convert_tokens_to_ids('[UNK]')
    valid_token_indices = token_ids != unk_token_id
    valid_token_embeddings = token_embeddings[0][valid_token_indices]

    if len(valid_token_embeddings) == 0:
        return torch.zeros((1, 768))  
    embedding = valid_token_embeddings.mean(dim=0)
    return embedding.cpu().numpy()

embeddings_list = []

for idx, row in final_reports_with_duplicates.iterrows():
    text = row['combined_text']
    
    embedding = get_embeddings_without_unk(text)
    
    embeddings_list.append({'subject_id': row['subject_id'], 'study_id': row['study_id'], 'embedding': embedding.flatten()})

embeddings_df = pd.DataFrame(embeddings_list)

embeddings_matrix = np.vstack(embeddings_df['embedding'].values) 
pca = PCA(n_components=256)
reduced_embeddings = pca.fit_transform(embeddings_matrix)

embeddings_df['reduced_embedding'] = list(reduced_embeddings)

print(embeddings_df.head())
embeddings_df.to_csv('final_reports_with_embeddings_reduced.csv', index=False)


  subject_id  study_id                                          embedding  \
0   10000032  50414267  [0.05278087, -0.29329956, 0.16665852, 0.116663...   
1   10000032  53189527  [0.0050405394, -0.2345086, 0.089080945, 0.1834...   
2   10000032  53189527  [0.0050405394, -0.2345086, 0.089080945, 0.1834...   
3   10000032  50414267  [0.05278087, -0.29329956, 0.16665852, 0.116663...   
4   10000764  57375967  [0.0328045, -0.14368281, 0.12894952, 0.1879166...   

                                   reduced_embedding  
0  [-1.2729756, -0.34996283, -0.4565846, 0.355985...  
1  [-1.3906875, -0.95247, -0.85298204, 0.4475414,...  
2  [-1.3906875, -0.9524744, -0.85298496, 0.447532...  
3  [-1.2729783, -0.34993517, -0.4566394, 0.355956...  
4  [-1.0518652, -0.7917509, 0.102976404, 0.149236...  


In [46]:
import shutil
import os

def delete_all_from_working_directory(directory_path):
    """
    Delete all files and folders in the specified directory.
    """
    if os.path.exists(directory_path):
        for item in os.listdir(directory_path):
            item_path = os.path.join(directory_path, item)
            try:
                if os.path.isfile(item_path) or os.path.islink(item_path):
                    os.unlink(item_path)  
                    print(f"Deleted file: {item_path}")
                elif os.path.isdir(item_path):
                    shutil.rmtree(item_path)  
                    print(f"Deleted folder: {item_path}")
            except Exception as e:
                print(f"Failed to delete {item_path}. Reason: {e}")
        print(f"All files and folders in {directory_path} have been deleted.")
    else:
        print(f"Directory does not exist: {directory_path}")

working_directory = './'  

delete_all_from_working_directory(working_directory)


Deleted file: ./final_reports_with_embeddings_reduced.csv
All files and folders in ./ have been deleted.


In [24]:
report_embeddings_df = pd.read_csv("/kaggle/input/report-embeddings/final_reports_with_embeddings_reduced.csv")
report_embeddings_df.head()

,subject_id,study_id,embedding,reduced_embedding
0,10000032,50414267,[ 5.27808703e-02 -2.93299556e-01 1.66658521e-...,[-1.27297556e+00 -3.49962831e-01 -4.56584603e-...
1,10000032,53189527,[ 5.04053943e-03 -2.34508604e-01 8.90809447e-...,[-1.39068747e+00 -9.52470005e-01 -8.52982044e-...
2,10000032,53189527,[ 5.04053943e-03 -2.34508604e-01 8.90809447e-...,[-1.39068747e+00 -9.52474415e-01 -8.52984965e-...
3,10000032,50414267,[ 5.27808703e-02 -2.93299556e-01 1.66658521e-...,[-1.27297831e+00 -3.49935174e-01 -4.56639409e-...
4,10000764,57375967,[ 3.28045003e-02 -1.43682808e-01 1.28949523e-...,[-1.05186522e+00 -7.91750908e-01 1.02976404e-...


In [60]:
report_embeddings_df.shape

(15115, 4)

In [25]:
image_embeddings_df = pd.read_csv("/kaggle/input/image-feature-embeddings/image_features.csv")
image_embeddings_df.head()

,subject_id,study_id,embedding
0,10000032.0,50414267.0,"[0.049698167, 0.0, 0.025857035, 0.0, 0.0524672..."
1,10000032.0,50414267.0,"[0.04969818, 0.0, 0.025856987, 0.0, 0.05246754..."
2,10000032.0,53189527.0,"[0.049699545, 0.0, 0.025857203, 0.0, 0.0524678..."
3,10000032.0,53189527.0,"[0.049699284, 0.0, 0.025856834, 0.0, 0.0524681..."
4,10000764.0,57375967.0,"[0.04969888, 0.0, 0.025856677, 0.0, 0.05246792..."


In [26]:
image_embeddings_df['subject_id'] = image_embeddings_df['subject_id'].astype(float).astype(int).astype(str)
image_embeddings_df['study_id'] = image_embeddings_df['study_id'].astype(float).astype(int).astype(str)

print(image_embeddings_df[['subject_id', 'study_id']].head())


  subject_id  study_id
0   10000032  50414267
1   10000032  50414267
2   10000032  53189527
3   10000032  53189527
4   10000764  57375967


In [27]:
image_embeddings_df.head()

,subject_id,study_id,embedding
0,10000032,50414267,"[0.049698167, 0.0, 0.025857035, 0.0, 0.0524672..."
1,10000032,50414267,"[0.04969818, 0.0, 0.025856987, 0.0, 0.05246754..."
2,10000032,53189527,"[0.049699545, 0.0, 0.025857203, 0.0, 0.0524678..."
3,10000032,53189527,"[0.049699284, 0.0, 0.025856834, 0.0, 0.0524681..."
4,10000764,57375967,"[0.04969888, 0.0, 0.025856677, 0.0, 0.05246792..."


In [50]:
def parse_embedding(embedding_str):
    embedding_str_cleaned = embedding_str.strip("[]").replace(",", "")
    embedding_values = np.array(embedding_str_cleaned.split(), dtype=np.float32)
    return embedding_values

report_embeddings = np.vstack(report_embeddings_df['reduced_embedding'].apply(parse_embedding).values)
image_embeddings = np.vstack(image_embeddings_df['embedding'].apply(parse_embedding).values)

concatenated_embeddings = np.concatenate((report_embeddings, image_embeddings), axis=1)

final_feature_df = pd.DataFrame()
final_feature_df['Concatenated_embeddings'] = [embedding.tolist() for embedding in concatenated_embeddings] 
final_feature_df['study_id'] = report_embeddings_df['study_id']  
final_feature_df['subject_id'] = report_embeddings_df['subject_id']  

final_feature_df.to_csv('final_combined_embeddings.csv', index=False)

print(final_feature_df.head())


                             Concatenated_embeddings  study_id  subject_id
0  [-1.2729755640029907, -0.34996283054351807, -0...  50414267    10000032
1  [-1.3906874656677246, -0.9524700045585632, -0....  53189527    10000032
2  [-1.3906874656677246, -0.9524744153022766, -0....  53189527    10000032
3  [-1.2729783058166504, -0.3499351739883423, -0....  50414267    10000032
4  [-1.0518652200698853, -0.7917509078979492, 0.1...  57375967    10000764


In [61]:
final_feature_df.shape

(15115, 3)

In [51]:
final_feature_df

,Concatenated_embeddings,study_id,subject_id
0,"[-1.2729755640029907, -0.34996283054351807, -0...",50414267,10000032
1,"[-1.3906874656677246, -0.9524700045585632, -0....",53189527,10000032
2,"[-1.3906874656677246, -0.9524744153022766, -0....",53189527,10000032
3,"[-1.2729783058166504, -0.3499351739883423, -0....",50414267,10000032
4,"[-1.0518652200698853, -0.7917509078979492, 0.1...",57375967,10000764
...,...,...,...
15110,"[11.666879653930664, -0.7274474501609802, -0.1...",59910967,10498557
15111,"[-0.9578160643577576, -1.0040777921676636, 0.4...",52457766,10498557
15112,"[-0.9578160643577576, -1.0040777921676636, 0.4...",52457766,10498557
15113,"[-1.0066105127334595, -0.10055671632289886, -0...",53217608,10498687


In [55]:
print(final_feature_df.columns) 
print(expanded_p10_sampled_df.columns)  


Index(['Concatenated_embeddings', 'study_id', 'subject_id'], dtype='object')
Index(['subject_id', 'study_id', 'Atelectasis', 'Cardiomegaly',
       'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture',
       'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion',
       'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices',
       'image_id'],
      dtype='object')


In [64]:
print(final_feature_df['study_id'].equals(expanded_p10_sampled_df['study_id']))
print(final_feature_df['subject_id'].equals(expanded_p10_sampled_df['subject_id']))


False
False


In [65]:
final_feature_df = final_feature_df.sort_values(by=['study_id', 'subject_id']).reset_index(drop=True)
expanded_p10_sampled_df = expanded_p10_sampled_df.sort_values(by=['study_id', 'subject_id']).reset_index(drop=True)


In [67]:
label_names = [
    "Atelectasis", "Cardiomegaly", "Consolidation", "Edema",
    "Enlarged Cardiomediastinum", "Fracture", "Lung Lesion",
    "Lung Opacity", "No Finding", "Pleural Effusion", "Pleural Other",
    "Pneumonia", "Pneumothorax", "Support Devices"
]

final_feature_df = pd.concat([final_feature_df, expanded_p10_sampled_df[label_names]], axis=1)


In [68]:
final_feature_df

,Concatenated_embeddings,study_id,subject_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,"[-0.7209290862083435, -1.5927101373672485, 1.0...",50003651,10076958,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[-0.7209290862083435, -1.5927101373672485, 1.0...",50003651,10076958,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[-0.7209290862083435, -1.5927101373672485, 1.0...",50003651,10076958,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[1.1479270458221436, 2.5772757530212402, -1.48...",50006246,10103748,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,1.0
4,"[-0.27433669567108154, 1.7107900381088257, -0....",50008565,10225882,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15110,"[-0.7999151349067688, 0.2899515926837921, -0.8...",59998484,10084069,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN
15111,"[-0.7999151349067688, 0.2899515926837921, -0.8...",59998484,10084069,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN
15112,"[-0.7986184358596802, -0.9371245503425598, 0.0...",59998528,10123063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
15113,"[-0.7986184358596802, -0.9371245503425598, 0.0...",59998528,10123063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0


In [70]:
final_feature_df = final_feature_df.sort_values(by='subject_id', ascending=True)
print(final_feature_df.head())

                                 Concatenated_embeddings  study_id  \
4703   [-1.3906874656677246, -0.9524700045585632, -0....  53189527   
4704   [-1.3906874656677246, -0.9524744153022766, -0....  53189527   
607    [-1.2729783058166504, -0.3499351739883423, -0....  50414267   
606    [-1.2729755640029907, -0.34996283054351807, -0...  50414267   
11206  [-1.0518652200698853, -0.7917509078979492, 0.1...  57375967   

       subject_id  Atelectasis  Cardiomegaly  Consolidation  Edema  \
4703     10000032          NaN           NaN            NaN    NaN   
4704     10000032          NaN           NaN            NaN    NaN   
607      10000032          NaN           NaN            NaN    NaN   
606      10000032          NaN           NaN            NaN    NaN   
11206    10000764          NaN           NaN            1.0    NaN   

       Enlarged Cardiomediastinum  Fracture  Lung Lesion  Lung Opacity  \
4703                          NaN       NaN          NaN           NaN   
4704      

In [71]:
final_feature_df = final_feature_df.reset_index(drop=True)
print(final_feature_df.head())

                             Concatenated_embeddings  study_id  subject_id  \
0  [-1.3906874656677246, -0.9524700045585632, -0....  53189527    10000032   
1  [-1.3906874656677246, -0.9524744153022766, -0....  53189527    10000032   
2  [-1.2729783058166504, -0.3499351739883423, -0....  50414267    10000032   
3  [-1.2729755640029907, -0.34996283054351807, -0...  50414267    10000032   
4  [-1.0518652200698853, -0.7917509078979492, 0.1...  57375967    10000764   

   Atelectasis  Cardiomegaly  Consolidation  Edema  \
0          NaN           NaN            NaN    NaN   
1          NaN           NaN            NaN    NaN   
2          NaN           NaN            NaN    NaN   
3          NaN           NaN            NaN    NaN   
4          NaN           NaN            1.0    NaN   

   Enlarged Cardiomediastinum  Fracture  Lung Lesion  Lung Opacity  \
0                         NaN       NaN          NaN           NaN   
1                         NaN       NaN          NaN           NaN

In [72]:
final_feature_df

,Concatenated_embeddings,study_id,subject_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,"[-1.3906874656677246, -0.9524700045585632, -0....",53189527,10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,"[-1.3906874656677246, -0.9524744153022766, -0....",53189527,10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,"[-1.2729783058166504, -0.3499351739883423, -0....",50414267,10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,"[-1.2729755640029907, -0.34996283054351807, -0...",50414267,10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,"[-1.0518652200698853, -0.7917509078979492, 0.1...",57375967,10000764,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15110,"[11.666879653930664, -0.7274474501609802, -0.1...",59910967,10498557,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
15111,"[-0.9578160643577576, -1.0040777921676636, 0.4...",52457766,10498557,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15112,"[-0.9578160643577576, -1.0040777921676636, 0.4...",52457766,10498557,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15113,"[-1.0066105127334595, -0.10055671632289886, -0...",53217608,10498687,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0


In [79]:


def mask_nan_values(tensor):
    """Mask NaN values in the tensor by replacing them with 0."""
    return tf.where(tf.math.is_nan(tensor), tf.zeros_like(tensor), tensor)

# Model Definition
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X.shape[1],)),  # Input layer
    tf.keras.layers.Dense(512, activation='relu'),          # First hidden layer
    tf.keras.layers.Dense(256, activation='relu'),          # Second hidden layer
    tf.keras.layers.Dense(128, activation='relu'),          # Third hidden layer
    tf.keras.layers.Dense(len(label_names), activation='sigmoid')  # Output layer
])

# Loss function for multi-label classification
def masked_binary_crossentropy(y_true, y_pred):
    """Mask NaN values in y_true and calculate binary crossentropy."""
    y_true = mask_nan_values(y_true)  # Apply mask to the true labels
    return tf.keras.losses.binary_crossentropy(y_true, y_pred)

# Compile the model
model.compile(optimizer='adam', loss=masked_binary_crossentropy, metrics=['accuracy'])

# Check NaN in data
print(f"Are there NaN values in X_train? {np.any(np.isnan(X_train))}")
print(f"Are there NaN values in y_train? {np.any(np.isnan(y_train))}")

# Step 1: Train the model using a custom training loop with masking
epochs = 20
batch_size = 32

# Step 2: Training loop
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for i in range(0, len(X_train), batch_size):
        # Get the batch data
        X_batch = X_train[i:i + batch_size]
        y_batch = y_train[i:i + batch_size]

        # Apply mask to the batch
        X_batch = np.array([mask_nan_values(x) for x in X_batch])
        y_batch = np.array([mask_nan_values(y) for y in y_batch])

        with tf.GradientTape() as tape:
            # Forward pass
            y_pred = model(X_batch, training=True)
            
            # Compute the loss
            loss = masked_binary_crossentropy(y_batch, y_pred)
        
        # Compute gradients and apply them
        grads = tape.gradient(loss, model.trainable_variables)
        model.optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Print the loss after each epoch
    print(f"Epoch {epoch+1} Loss: {loss.numpy()}")

# Step 3: Evaluate the model after training
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

# Optionally, save the model
# model.save('masked_fc_fnn_model.h5')


Are there NaN values in X_train? False
Are there NaN values in y_train? True
Epoch 1/20
Epoch 1 Loss: [3.2637689e-02 4.8182136e-01 5.1948242e-02 3.2373205e-02 2.3739759e-03
 1.7392892e-02 1.6918418e-01 4.4715730e-03 1.1545841e-01 6.1638024e-02
 1.2903738e-01 4.4288015e-01 2.9697657e-01 6.4692855e-02 1.8973933e-01
 4.0946625e-02 7.3648132e-02 1.3466461e-01 3.5996017e-01 1.3850722e-01
 2.8719958e-02 3.9105018e-04 2.7343288e-01 1.0432781e-01 2.5811007e-03
 9.1949448e-02 3.1345013e-01 4.9040931e-01]
Epoch 2/20
Epoch 2 Loss: [2.3592489e-02 5.1723599e-01 4.3874774e-02 1.1680412e-02 8.0758042e-04
 6.4260853e-03 1.7271709e-01 1.8945491e-03 7.1352884e-02 1.7403843e-02
 1.2528063e-01 4.2580482e-01 2.9289177e-01 6.1862197e-02 1.0272581e-01
 1.6318586e-02 8.3265409e-02 7.1707986e-02 3.8330832e-01 1.1937533e-01
 1.9485448e-02 1.5905469e-04 1.3180242e-01 7.0290439e-02 6.1677053e-04
 5.8154650e-02 3.0727938e-01 3.2380125e-01]
Epoch 3/20
Epoch 3 Loss: [2.15037074e-02 4.50972229e-01 3.36790048e-02 7.92

In [85]:
X = np.vstack(final_feature_df['Concatenated_embeddings'].apply(np.array).values)
y = final_feature_df[label_names].values  

def mask_nan_values(tensor):
    """Mask NaN values and convert to zero."""
    mask = tf.math.is_nan(tensor)
    tensor = tf.where(mask, tf.zeros_like(tensor), tensor)  
    return tensor

def masked_binary_crossentropy(y_true, y_pred):
    """Mask NaN values in y_true and calculate binary crossentropy."""
    y_true = mask_nan_values(y_true) 
    loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)  
    return tf.reduce_mean(loss)  

def create_model():
    model = tf.keras.Sequential([
        layers.InputLayer(input_shape=(X.shape[1],)), 
        layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)),  
        layers.Dropout(0.5),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),  
        layers.Dropout(0.5),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)), 
        layers.Dropout(0.5),
        layers.Dense(len(label_names), activation='sigmoid')  
    ])
    model.compile(optimizer=optimizers.Adam(), loss=masked_binary_crossentropy, metrics=['accuracy'])
    return model
    
kf = KFold(n_splits=10, shuffle=True, random_state=42)
fold = 1
for train_index, val_index in kf.split(X):
    print(f"Training fold {fold}...")
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    model = create_model()
    lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)
    model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val), callbacks=[lr_scheduler])

    val_loss, val_acc = model.evaluate(X_val, y_val)
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")
    fold += 1
    model.save(f'fc_fnn_model_fold_{fold}.h5')


Training fold 1...
Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


426/426 ━━━━━━━━━━━━━━━━━━━━ 13s 15ms/step - accuracy: 0.1851 - loss: 4.8272 - val_accuracy: 0.1415 - val_loss: 0.4711 - learning_rate: 0.0010
Epoch 2/30
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1978 - loss: 0.4290 - val_accuracy: 0.2110 - val_loss: 0.4060 - learning_rate: 0.0010
Epoch 3/30
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1955 - loss: 0.4200 - val_accuracy: 0.2110 - val_loss: 0.3777 - learning_rate: 0.0010
Epoch 4/30
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2017 - loss: 0.4140 - val_accuracy: 0.1316 - val_loss: 0.3607 - learning_rate: 0.0010
Epoch 5/30
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2006 - loss: 0.3947 - val_accuracy: 0.1349 - val_loss: 0.3463 - learning_rate: 0.0010
Epoch 6/30
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2071 - loss: 0.3853 - val_accuracy: 0.1991 - val_loss: 0.3337 - learning_rate: 0.0010
Epoch 7/30
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2165 - loss: 0.3688 - val_accur